In [1]:
import aspose.slides as slides
import os
import shutil
from pydub import AudioSegment
import whisper
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

AudioSegment.converter = "C:\\PATH_Programs\\ffmpeg.exe"
AudioSegment.ffmpeg = "C:\\PATH_Programs\\ffmpeg.exe"
AudioSegment.ffprobe ="C:\\PATH_Programs\\ffprobe.exe"

In [2]:
output_folder = "audio_files"

# Delete the output folder if it exists so it empties
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

# Create the output folder if it doesn't exist (should run everytime)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
#input powerpoint presentation name
presentation_name = "presentations/INSERTNAMEHERE.pptx"
parts = presentation_name.split('.')
part_without_extension = parts[0]
final_name = part_without_extension + ".pdf"

In [4]:
from aspose.slides import AudioFrame

with slides.Presentation(presentation_name) as presentation:
    for index, audio in enumerate(presentation.audios):
        audio_data = audio.binary_data
        content_type = audio.content_type
        file_extension = content_type.split("/")[-1]  # Get the file extension from content type
        
        # Construct the file name
        file_name = f"audio_{index + 1}.{file_extension}"
        
        # Write the binary audio data to a file
        with open(os.path.join(output_folder, file_name), "wb") as file:
            file.write(audio_data)
        
        print(f"Downloaded {file_name} to {output_folder}")

In [6]:
#"tiny" = 32x speed - worst accuracy
#"base" = 16x speed - recommended
#"small" = 6x speed - best accuracy
model_type = "small"

In [7]:
model = whisper.load_model(model_type)
results = []
audio_files = [f for f in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, f))]
for file in audio_files:
    result = model.transcribe(os.path.join(output_folder,file))
    res_text = result.get("text")
    results.append(res_text)

In [8]:
def create_pdf(results, output_file):
    # Create a PDF document
    doc = SimpleDocTemplate(output_file, pagesize=letter)

    # Define styles for paragraphs
    styles = getSampleStyleSheet()
    style = styles["Normal"]

    # Add each result as a paragraph to the document
    paragraphs = []
    for result in results:
        # Add the result as a paragraph to the document
        p = Paragraph(result.strip(), style)
        paragraphs.append(p)
        
        # Add a single line break to separate items
        paragraphs.append(Spacer(1, 12))  # Adjust the second argument to control the spacing

    # Build the PDF document
    doc.build(paragraphs)

In [9]:
output_file = final_name
create_pdf(results, output_file)
print(f"PDF created: {output_file}")

PDF created: Lecture 4_History part 1.pdf
